# Creating HTTP server using http library

This file explains how to create http server.
You should use 9-http-client.ipynb to connect to this server as client.

**NOTE**: Last cell in this doc will stop the server.

In [3]:
const http = require('http');
const url = require('url');
const fs = require('fs');
const path = require('path');
const mime = require('mime-types');

## Adding some helper functions and variables

In [4]:
const cwd = process.cwd(); // Gets the current working directory
/**
 * Determines the MIME type for a given file based on its file extension.
 * 
 * @param {string} filePath - The path to the file whose MIME type is to be determined.
 * @returns {string} The MIME type of the file. If the file extension is not recognized, returns 'application/octet-stream'.
 * 
 * @description
 * This function extracts the file extension from the provided `filePath`, converts it to lowercase, and looks up the corresponding MIME type using the `mime` library.
 * If the MIME type for the given file extension is not found, it defaults to 'application/octet-stream'.
 * 
 * @example
 * const mimeType = getContentType('/path/to/file.jpg');
 * console.log(mimeType); // Outputs: 'image/jpeg'
 */
function getContentType(filePath) {
    const extname = path.extname(filePath).toLowerCase();
    return mime.lookup(extname) || 'application/octet-stream';
}

## Creating GET, POST and PUT handler

In [5]:
/**
 * Handles HTTP GET requests based on the request pathname.
 * 
 * @param {string} pathname - The pathname extracted from the URL of the request.
 * @param {object} query - The query parameters parsed from the URL.
 * @param {http.IncomingMessage} req - The request object, containing details about the HTTP request.
 * @param {http.ServerResponse} res - The response object, used to send a response back to the client.
 * 
 * @description
 * This function handles GET requests for different paths:
 * - `/json-response/`: Reads a JSON file from the server, adds a timestamp, and sends the modified JSON as the response.
 * - `/download-data/`: Reads a JSON file from the server and sends it as the response.
 * - `/download-image/`: Streams an image file from the server to the client.
 * - For any other path: Responds with a plain text message including the pathname and query parameters.
 * 
 * @example
 * // Example usage:
 * handleGET('/json-response/', { userId: '123' }, req, res);
 * 
 * @throws {Error} Throws an error if there is an issue reading the file or streaming the image.
 */
function handleGET(pathname, query, req, res) {
    switch (pathname.toLowerCase()) {
        case '/json-response/':
            const fileAbsolutePath = path.join(cwd, 'http-server-downloads', 'server-download-data.json');
            // Reads data from file as text ('utf8')
            fs.readFile(fileAbsolutePath, 'utf8', (error, data) => {
                if (error) { 
                    res.setHeader('Content-Type', 'text/plain'); // Set header for error response
                    res.statusCode = 500; // Set status code for server error
                    res.end('An error occurred during getting file content.');
                } else {
                    res.setHeader('Content-Type', 'application/json'); // Set header for JSON response
                    res.statusCode = 200; // Set status code for successful response
                    const parsedData = JSON.parse(data); // Parse JSON data
                    parsedData.responseTimestamp = Date.now(); // Add timestamp
                    res.end(JSON.stringify(parsedData)); // Send modified JSON as string
                }
            });
            break;
        case '/download-data/':
            const downloadFileAbsolutePath = path.join(cwd, 'http-server-downloads', 'server-download-data.json');
            // Reads data from file as buffer
            fs.readFile(downloadFileAbsolutePath, (error, data) => {
                if (error) { 
                    res.setHeader('Content-Type', 'text/plain'); // Set header for error response
                    res.statusCode = 500; // Set status code for server error
                    res.end('An error occurred during getting file content.');
                } else {
                    res.setHeader('Content-Type', 'application/json'); // Set header for JSON response
                    res.statusCode = 200; // Set status code for successful response
                    res.end(data); // Send file content as binary data
                }
            });
            break;
        case '/download-image/':
            const imageAbsolutePath = path.join(cwd, 'http-server-downloads', 'server-download-image.png');
            const imageType = getContentType(imageAbsolutePath); // Get MIME type of image

            try {
                const imageStream = fs.createReadStream(imageAbsolutePath);
                
                res.setHeader('Content-Type', imageType || 'image/jpeg'); // Set content type for image
                res.statusCode = 200; // Set status code for successful response
                
                imageStream.on('error', (err) => {
                    console.error('Error reading image file:', err);
                    res.setHeader('Content-Type', 'text/plain'); // Set header for error response
                    res.statusCode = 500; // Set status code for server error
                    res.end('An error occurred while reading the image file.');
                });

                imageStream.pipe(res); // Pipe image stream to response

            } catch (error) {
                console.error('Error handling download-image request:', error);
                res.setHeader('Content-Type', 'text/plain'); // Set header for error response
                res.statusCode = 500; // Set status code for server error
                res.end('An error occurred while serving the image.');
            }
            break;
        default:
            // Handle unknown paths
            res.statusCode = 200; // Set status code for successful response
            res.setHeader('Content-Type', 'text/plain'); // Set header for plain text response

            res.write('Hello World ');
            res.write(`pathname: "${pathname}", `);
            res.write(`query: "${JSON.stringify(query)}", `);

            res.end(); // End response
    }
}

In [6]:
function handlePOST(pathname, req, res) {
    
}

In [7]:
/**
 * Handles HTTP PUT requests for uploading files.
 * 
 * @param {string} pathname - The pathname extracted from the request URL.
 * @param {http.IncomingMessage} req - The request object, containing the file data to be uploaded.
 * @param {http.ServerResponse} res - The response object used to send a response back to the client.
 * 
 * @description
 * This function handles PUT requests for file uploads in two ways based on the pathname:
 * 
 * - **`/stream/`**: Uses streaming to handle file uploads. The request data is piped directly into a writable stream, which writes the data to the file on disk.
 * - **`/direct/`**: Collects data chunks from the request, concatenates them, and writes the complete data to a file in one go.
 * 
 * If the pathname does not match either of these patterns, a 404 Not Found response is sent.
 * 
 * **Error Handling**:
 * - Errors during file writing or streaming result in a 500 Internal Server Error response.
 * - Errors with the request stream also result in a 500 Internal Server Error response.
 * 
 * @example
 * // Example usage for streaming upload:
 * handlePUT('/stream/upload', req, res);
 * 
 * // Example usage for direct upload:
 * handlePUT('/direct/upload', req, res);
 */
function handlePUT(pathname, req, res) {
    const uploadDir = path.join(cwd, 'http-server-uploads');
    const uploadFilePath = path.join(cwd, 'http-server-uploads', 'uploaded-image.png');

    // Create the upload directory if it does not exist
    if (!fs.existsSync(uploadDir)) {
        fs.mkdirSync(uploadDir, { recursive: true });
    }

    if (pathname.startsWith('/stream/')) {
        // Create a writable stream to save the file
        const fileStream = fs.createWriteStream(uploadFilePath);

        // Pipe the request data to the file stream
        req.pipe(fileStream);

        // Handle the completion of the file writing process; file is fully written.
        fileStream.on('finish', () => {
            res.statusCode = 200;
            res.setHeader('Content-Type', 'text/plain');
            res.end('File uploaded and saved successfully.');
        });

        // Handle any errors during the file writing process
        fileStream.on('error', (err) => {
            console.error('Error writing file:', err);
            res.statusCode = 500;
            res.setHeader('Content-Type', 'text/plain');
            res.end('Internal Server Error.');
        });

        // Handle errors from the request stream
        req.on('error', (err) => {
            console.error('Error with request:', err);
            res.statusCode = 500;
            res.setHeader('Content-Type', 'text/plain');
            res.end('Internal Server Error.');
        });

    } else if (pathname.startsWith('/direct/')) { // Expects binary data
        // Collect binary data and write it directly
        const chunks = [];

        req.on('data', (chunk) => {
            chunks.push(chunk);
        });

        req.on('end', () => {
            // Concatenate all chunks and write to file
            const buffer = Buffer.concat(chunks);
            fs.writeFile(uploadFilePath, buffer, (err) => {
                if (err) {
                    console.error('Error writing file:', err);
                    res.statusCode = 500;
                    res.setHeader('Content-Type', 'text/plain');
                    res.end('Internal Server Error.');
                    return;
                }
                res.statusCode = 200;
                res.setHeader('Content-Type', 'text/plain');
                res.end('File uploaded and saved successfully.');
            });
        });

        req.on('error', (err) => {
            console.error('Error with request:', err);
            res.statusCode = 500;
            res.setHeader('Content-Type', 'text/plain');
            res.end('Internal Server Error.');
        });
    } else {
        res.statusCode = 404;
        res.setHeader('Content-Type', 'text/plain');
        res.end('Not Found.');
    }
}

## Initiating the server

In [8]:
/**
 * Creates an HTTP server and handles incoming requests based on their method.
 * 
 * @param {http.IncomingMessage} req - The request object, containing details about the HTTP request.
 * @param {http.ServerResponse} res - The response object, used to send a response back to the client.
 * 
 * @function
 * @description 
 * The server parses the URL of incoming requests and determines the appropriate action based on the HTTP method.
 * - For `GET` requests, it calls `handleGET`.
 * - For `POST` requests, it calls `handlePOST`.
 * - For `PUT` requests, it calls `handlePUT`.
 * - For other methods, it responds with a 405 status code and a 'Not allowed' message.
 * 
 * 
 * @returns {http.Server} The created HTTP server instance.
 */
const server = http.createServer((req, res) => {
    const parsedUrl = url.parse(req.url, true); // Parse the URL with query string support
    const pathname = parsedUrl.pathname;
    const query = parsedUrl.query; // as object

    // Access headers
    const headers = req.headers;
    console.log('Request headers:', headers);
    
    switch (req.method.toUpperCase()) {
        case 'GET':
            handleGET(pathname, query, req, res);
            break;
        case 'POST':
            handlePOST(pathname, req, res);
            break;
        case 'PUT':
            handlePUT(pathname, req, res);
            break;
        default:
            res.statusCode = 405; // Set status code
            res.setHeader('Content-Type', 'text/plain'); // Set header
            res.end('Not allowed.');
    }
});

## Listening the port

In [9]:
// Starts the server and listens to the given port
server.listen(1234, () => {
    console.log('Server started on port 1234.');
})

<ref *1> Server {
  maxHeaderSize: undefined,
  insecureHTTPParser: undefined,
  requestTimeout: 300000,
  headersTimeout: 60000,
  keepAliveTimeout: 5000,
  connectionsCheckingInterval: 30000,
  joinDuplicateHeaders: undefined,
  _events: [Object: null prototype] {
    request: [Function (anonymous)],
    connection: [Function: connectionListener],
    listening: [Function: bound onceWrapper] { listener: [Function (anonymous)] }
  },
  _eventsCount: 3,
  _maxListeners: undefined,
  _connections: 0,
  _handle: TCP {
    reading: false,
    onconnection: [Function: onconnection],
    [Symbol(owner_symbol)]: [Circular *1]
  },
  _usingWorkers: false,
  _workers: [],
  _unref: false,
  allowHalfOpen: true,
  pauseOnConnect: false,
  noDelay: true,
  keepAlive: false,
  keepAliveInitialDelay: 0,
  httpAllowHalfOpen: false,
  timeout: 0,
  maxHeadersCount: null,
  maxRequestsPerSocket: 0,
  _connectionKey: '6::::1234',
  [Symbol(IncomingMessage)]: [Function: IncomingMessage],
  [Symbol(Serv

Server started on port 1234.
Request headers: {
  host: 'localhost:1234',
  connection: 'keep-alive',
  accept: '*/*',
  'accept-language': '*',
  'sec-fetch-mode': 'cors',
  'user-agent': 'undici',
  'accept-encoding': 'gzip, deflate'
}
Request headers: {
  accept: 'application/json, text/plain, */*',
  'user-agent': 'axios/1.7.4',
  'accept-encoding': 'gzip, compress, deflate, br',
  host: 'localhost:1234',
  connection: 'close'
}
Request headers: {
  host: 'localhost:1234',
  connection: 'keep-alive',
  accept: '*/*',
  'accept-language': '*',
  'sec-fetch-mode': 'cors',
  'user-agent': 'undici',
  'accept-encoding': 'gzip, deflate'
}
Request headers: {
  accept: 'application/json, text/plain, */*',
  'user-agent': 'axios/1.7.4',
  'accept-encoding': 'gzip, compress, deflate, br',
  host: 'localhost:1234',
  connection: 'close'
}
Request headers: {
  host: 'localhost:1234',
  connection: 'keep-alive',
  accept: '*/*',
  'accept-language': '*',
  'sec-fetch-mode': 'cors',
  'user-age

## Stopping the server

In [10]:
// Closes the server
server.close((err) => {
    if (err) {
        console.error('Error stopping the server:', err);
        return;
    }
    console.log('Server stopped.');
});

Server stopped.
